<a href="https://colab.research.google.com/github/Reshma2103/reshma/blob/main/flight_price.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report,confusion_matrix
import warnings
import pickle
from scipy import stats
warnings.filterwarnings('ignore')
plt.style.use('fivethirtyeight')


In [ ]:
data=pd.read_excel('/content/Data_Train.xlsx')
data.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [ ]:
data.isnull().sum()

Airline            0
Date_of_Journey    0
Source             0
Destination        0
Route              1
Dep_Time           0
Arrival_Time       0
Duration           0
Total_Stops        1
Additional_Info    0
Price              0
dtype: int64

In [ ]:
data.dropna(inplace=True)

In [ ]:
data.Date_of_Journey=data.Date_of_Journey.str.split('/')

In [ ]:
data.Date_of_Journey

0        [24, 03, 2019]
1         [1, 05, 2019]
2         [9, 06, 2019]
3        [12, 05, 2019]
4        [01, 03, 2019]
              ...      
10678     [9, 04, 2019]
10679    [27, 04, 2019]
10680    [27, 04, 2019]
10681    [01, 03, 2019]
10682     [9, 05, 2019]
Name: Date_of_Journey, Length: 10682, dtype: object

In [ ]:
data['Date']=data.Date_of_Journey.str[0]
data['Month']=data.Date_of_Journey.str[1]
data['Year']=data.Date_of_Journey.str[2]

In [ ]:
data.Total_Stops.unique()

array(['non-stop', '2 stops', '1 stop', '3 stops', '4 stops'],
      dtype=object)

In [ ]:
data.Route=data.Route.str.split('→')

In [ ]:
data.Route

0                      [BLR ,  DEL]
1        [CCU ,  IXR ,  BBI ,  BLR]
2        [DEL ,  LKO ,  BOM ,  COK]
3               [CCU ,  NAG ,  BLR]
4               [BLR ,  NAG ,  DEL]
                    ...            
10678                  [CCU ,  BLR]
10679                  [CCU ,  BLR]
10680                  [BLR ,  DEL]
10681                  [BLR ,  DEL]
10682    [DEL ,  GOI ,  BOM ,  COK]
Name: Route, Length: 10682, dtype: object

In [ ]:
data['city1']=data.Route.str[0]
data['city2']=data.Route.str[1]
data['city3']=data.Route.str[2]
data['city4']=data.Route.str[3]
data['city5']=data.Route.str[4]
data['city6']=data.Route.str[5]

In [ ]:
data.Dep_Time=data.Dep_Time.str.split(':')

In [ ]:
data['Dep_Time_Hour']=data.Dep_Time.str[0]
data['Dep_Time_Mins']=data.Dep_Time.str[1]

In [ ]:
data.Arrival_Time=data.Arrival_Time.str.split(':')

In [ ]:
data['Arrival_Time_Hour']=data.Arrival_Time.str[0]
data['Arrival_Time_Mins']=data.Arrival_Time.str[1]

In [ ]:
data.Arrival_Time_Mins=data.Arrival_Time_Mins.str.split(' ')
data['Arrival_Time_Mins']=data.Arrival_Time_Mins.str[0]
data['Arrival_Day']=data.Arrival_Time_Mins.str[1]

In [ ]:
data.Duration=data.Duration.str.split(' ')

In [ ]:
data['Travel_Hours']=data.Duration.str[0]
data['Travel_Hours']=data['Travel_Hours'].str.split('h')
data['Travel_Hours']=data['Travel_Hours'].str[0]
data.Travel_Hours=data.Travel_Hours
data['Travel_Mins']=data.Duration.str[1]

In [ ]:
data['Travel_Mins']=data['Travel_Mins'].str.split('m')
data['Travel_Mins']=data['Travel_Mins'].str[0]

In [ ]:
data.Total_Stops=data.Total_Stops.str.split(' ')
data.Total_Stops=data.Total_Stops.str[0]
data.Total_Stops.replace('non-stop',0,inplace=True)

In [ ]:
data.Total_Stops

0        0
1        2
2        2
3        1
4        1
        ..
10678    0
10679    0
10680    0
10681    0
10682    2
Name: Total_Stops, Length: 10682, dtype: object

In [ ]:
data.Additional_Info.unique()

array(['No info', 'In-flight meal not included',
       'No check-in baggage included', '1 Short layover', 'No Info',
       '1 Long layover', 'Change airports', 'Business class',
       'Red-eye flight', '2 Long layover'], dtype=object)

In [ ]:
data.Additional_Info.replace('No Info','No info',inplace=True)

In [ ]:
data.isnull().sum()

Airline                  0
Date_of_Journey          0
Source                   0
Destination              0
Route                    0
Dep_Time                 0
Arrival_Time             0
Duration                 0
Total_Stops              0
Additional_Info          0
Price                    0
Date                     0
Month                    0
Year                     0
city1                    0
city2                    0
city3                 3491
city4                 9116
city5                10636
city6                10681
Dep_Time_Hour            0
Dep_Time_Mins            0
Arrival_Time_Hour        0
Arrival_Time_Mins        0
Arrival_Day              0
Travel_Hours             0
Travel_Mins           1032
dtype: int64

In [ ]:
data.drop(['city4','city5','city6'],axis=1,inplace=True)

In [ ]:
data.isnull().sum()

Airline                 0
Date_of_Journey         0
Source                  0
Destination             0
Route                   0
Dep_Time                0
Arrival_Time            0
Duration                0
Total_Stops             0
Additional_Info         0
Price                   0
Date                    0
Month                   0
Year                    0
city1                   0
city2                   0
city3                3491
Dep_Time_Hour           0
Dep_Time_Mins           0
Arrival_Time_Hour       0
Arrival_Time_Mins       0
Arrival_Day             0
Travel_Hours            0
Travel_Mins          1032
dtype: int64

In [ ]:
data.drop(['Date_of_Journey','Route','Dep_Time','Arrival_Time','Duration'],axis=1,inplace=True)

In [ ]:
data.isnull().sum()

Airline                 0
Source                  0
Destination             0
Total_Stops             0
Additional_Info         0
Price                   0
Date                    0
Month                   0
Year                    0
city1                   0
city2                   0
city3                3491
Dep_Time_Hour           0
Dep_Time_Mins           0
Arrival_Time_Hour       0
Arrival_Time_Mins       0
Arrival_Day             0
Travel_Hours            0
Travel_Mins          1032
dtype: int64

In [ ]:
data['Arrival_Day'].fillna(data['Date'],inplace=True)

In [ ]:
data['city3'].fillna('None',inplace=True)

In [ ]:
data['Travel_Mins'].fillna(0,inplace=True)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10682 entries, 0 to 10682
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Airline            10682 non-null  object
 1   Source             10682 non-null  object
 2   Destination        10682 non-null  object
 3   Total_Stops        10682 non-null  object
 4   Additional_Info    10682 non-null  object
 5   Price              10682 non-null  int64 
 6   Date               10682 non-null  object
 7   Month              10682 non-null  object
 8   Year               10682 non-null  object
 9   city1              10682 non-null  object
 10  city2              10682 non-null  object
 11  city3              10682 non-null  object
 12  Dep_Time_Hour      10682 non-null  object
 13  Dep_Time_Mins      10682 non-null  object
 14  Arrival_Time_Hour  10682 non-null  object
 15  Arrival_Time_Mins  10682 non-null  object
 16  Arrival_Day        10682 non-null  objec

In [ ]:
data.head(3)

,Airline,Source,Destination,Total_Stops,Additional_Info,Price,Date,Month,Year,city1,city2,city3,Dep_Time_Hour,Dep_Time_Mins,Arrival_Time_Hour,Arrival_Time_Mins,Arrival_Day,Travel_Hours,Travel_Mins
0,IndiGo,Banglore,New Delhi,0,No info,3897,24,03,2019,BLR,DEL,None,22,20,01,10,0,2,50
1,Air India,Kolkata,Banglore,2,No info,7662,1,05,2019,CCU,IXR,BBI,05,50,13,15,5,7,25
2,Jet Airways,Delhi,Cochin,2,No info,13882,9,06,2019,DEL,LKO,BOM,09,25,04,25,5,19,0


In [ ]:
data.Total_Stops=data.Total_Stops.astype('int64')
data.Date=data.Date.astype('int64')
data.Month=data.Month.astype('int64')
data.Year=data.Year.astype('int64')
data.Dep_Time_Hour=data.Dep_Time_Hour.astype('int64')
data.Dep_Time_Mins=data.Dep_Time_Mins.astype('int64')
data.Arrival_Time_Hour=data.Arrival_Time_Hour.astype('int64')
data.Travel_Mins=data.Travel_Mins.astype('int64')

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10682 entries, 0 to 10682
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Airline            10682 non-null  object
 1   Source             10682 non-null  object
 2   Destination        10682 non-null  object
 3   Total_Stops        10682 non-null  int64 
 4   Additional_Info    10682 non-null  object
 5   Price              10682 non-null  int64 
 6   Date               10682 non-null  int64 
 7   Month              10682 non-null  int64 
 8   Year               10682 non-null  int64 
 9   city1              10682 non-null  object
 10  city2              10682 non-null  object
 11  city3              10682 non-null  object
 12  Dep_Time_Hour      10682 non-null  int64 
 13  Dep_Time_Mins      10682 non-null  int64 
 14  Arrival_Time_Hour  10682 non-null  int64 
 15  Arrival_Time_Mins  10682 non-null  object
 16  Arrival_Day        10682 non-null  objec

In [ ]:
data.Arrival_Time_Mins=data.Arrival_Time_Mins.astype('int64')

In [ ]:
data.Arrival_Day=data.Arrival_Day.astype('int64')

In [ ]:
data[data['Travel_Hours']=='5m']

,Airline,Source,Destination,Total_Stops,Additional_Info,Price,Date,Month,Year,city1,city2,city3,Dep_Time_Hour,Dep_Time_Mins,Arrival_Time_Hour,Arrival_Time_Mins,Arrival_Day,Travel_Hours,Travel_Mins
6474,Air India,Mumbai,Hyderabad,2,No info,17327,6,3,2019,BOM,GOI,PNQ,16,50,16,55,5,5m,0


In [ ]:
data.drop(index=6474,inplace=True,axis=0)

KeyError: ignored

In [ ]:
data.Travel_Hours=data.Travel_Hours.astype('int64')

In [ ]:
column=[column for column in data.columns if data[column].dtype=='object']
column

['Airline',
 'Source',
 'Destination',
 'Additional_Info',
 'city1',
 'city2',
 'city3']

In [ ]:
continuous_col =[column for column in data.columns if data[column].dtype!='objects']
continuous_col

['Airline',
 'Source',
 'Destination',
 'Total_Stops',
 'Additional_Info',
 'Price',
 'Date',
 'Month',
 'Year',
 'city1',
 'city2',
 'city3',
 'Dep_Time_Hour',
 'Dep_Time_Mins',
 'Arrival_Time_Hour',
 'Arrival_Time_Mins',
 'Arrival_Day',
 'Travel_Hours',
 'Travel_Mins']

In [ ]:
categorical = data[column]
categorical

,Airline,Source,Destination,Additional_Info,city1,city2,city3
0,IndiGo,Banglore,New Delhi,No info,BLR,DEL,None
1,Air India,Kolkata,Banglore,No info,CCU,IXR,BBI
2,Jet Airways,Delhi,Cochin,No info,DEL,LKO,BOM
3,IndiGo,Kolkata,Banglore,No info,CCU,NAG,BLR
4,IndiGo,Banglore,New Delhi,No info,BLR,NAG,DEL
...,...,...,...,...,...,...,...
10678,Air Asia,Kolkata,Banglore,No info,CCU,BLR,None
10679,Air India,Kolkata,Banglore,No info,CCU,BLR,None
10680,Jet Airways,Banglore,Delhi,No info,BLR,DEL,None
10681,Vistara,Banglore,New Delhi,No info,BLR,DEL,None


In [57]:
numerical=data[continuous_col]

In [59]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

In [67]:
data.Airline=le.fit_transform(data.Airline)
data.Source=le.fit_transform(data.Source)
data.Destination=le.fit_transform(data.Destination)
data.Additional_Info=le.fit_transform(data.Additional_Info)
data.city1=le.fit_transform(data.city1)
data.city2=le.fit_transform(data.city2)
data.city3=le.fit_transform(data.city3)
data.head()


,Airline,Source,Destination,Total_Stops,Additional_Info,Price,Date,Month,Year,city1,city2,city3,Dep_Time_Hour,Dep_Time_Mins,Arrival_Time_Hour,Arrival_Time_Mins,Arrival_Day,Travel_Hours,Travel_Mins
0,3,0,5,0,7,3897,24,3,2019,0,13,29,22,20,1,10,0,2,50
1,1,3,0,2,7,7662,1,5,2019,2,25,1,5,50,13,15,5,7,25
2,4,2,1,2,7,13882,9,6,2019,3,32,4,9,25,4,25,5,19,0
3,3,3,0,1,7,6218,12,5,2019,2,34,3,18,5,23,30,0,5,25
4,3,0,5,1,7,13302,1,3,2019,0,34,8,16,50,21,35,5,4,45


In [68]:
categorical

,Airline,Source,Destination,Additional_Info,city1,city2,city3
0,IndiGo,Banglore,New Delhi,No info,BLR,DEL,None
1,Air India,Kolkata,Banglore,No info,CCU,IXR,BBI
2,Jet Airways,Delhi,Cochin,No info,DEL,LKO,BOM
3,IndiGo,Kolkata,Banglore,No info,CCU,NAG,BLR
4,IndiGo,Banglore,New Delhi,No info,BLR,NAG,DEL
...,...,...,...,...,...,...,...
10678,Air Asia,Kolkata,Banglore,No info,CCU,BLR,None
10679,Air India,Kolkata,Banglore,No info,CCU,BLR,None
10680,Jet Airways,Banglore,Delhi,No info,BLR,DEL,None
10681,Vistara,Banglore,New Delhi,No info,BLR,DEL,None


In [69]:
fdata=data.copy()

In [70]:
fdata

,Airline,Source,Destination,Total_Stops,Additional_Info,Price,Date,Month,Year,city1,city2,city3,Dep_Time_Hour,Dep_Time_Mins,Arrival_Time_Hour,Arrival_Time_Mins,Arrival_Day,Travel_Hours,Travel_Mins
0,3,0,5,0,7,3897,24,3,2019,0,13,29,22,20,1,10,0,2,50
1,1,3,0,2,7,7662,1,5,2019,2,25,1,5,50,13,15,5,7,25
2,4,2,1,2,7,13882,9,6,2019,3,32,4,9,25,4,25,5,19,0
3,3,3,0,1,7,6218,12,5,2019,2,34,3,18,5,23,30,0,5,25
4,3,0,5,1,7,13302,1,3,2019,0,34,8,16,50,21,35,5,4,45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10678,0,3,0,0,7,4107,9,4,2019,2,5,29,19,55,22,25,5,2,30
10679,1,3,0,0,7,4145,27,4,2019,2,5,29,20,45,23,20,0,2,35
10680,4,0,2,0,7,7229,27,4,2019,0,13,29,8,20,11,20,0,3,0
10681,10,0,5,0,7,12648,1,3,2019,0,13,29,11,30,14,10,0,2,40


In [72]:
fdata.drop(['Additional_Info','city1','city2','city3'],axis=1,inplace=True)
fdata

,Airline,Source,Destination,Total_Stops,Price,Date,Month,Year,Dep_Time_Hour,Dep_Time_Mins,Arrival_Time_Hour,Arrival_Time_Mins,Arrival_Day,Travel_Hours,Travel_Mins
0,3,0,5,0,3897,24,3,2019,22,20,1,10,0,2,50
1,1,3,0,2,7662,1,5,2019,5,50,13,15,5,7,25
2,4,2,1,2,13882,9,6,2019,9,25,4,25,5,19,0
3,3,3,0,1,6218,12,5,2019,18,5,23,30,0,5,25
4,3,0,5,1,13302,1,3,2019,16,50,21,35,5,4,45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10678,0,3,0,0,4107,9,4,2019,19,55,22,25,5,2,30
10679,1,3,0,0,4145,27,4,2019,20,45,23,20,0,2,35
10680,4,0,2,0,7229,27,4,2019,8,20,11,20,0,3,0
10681,10,0,5,0,12648,1,3,2019,11,30,14,10,0,2,40


In [74]:
fdata.drop(['Total_Stops','Travel_Hours','Travel_Mins'],axis=1,inplace=True)
fdata

,Airline,Source,Destination,Price,Date,Month,Year,Dep_Time_Hour,Dep_Time_Mins,Arrival_Time_Hour,Arrival_Time_Mins,Arrival_Day
0,3,0,5,3897,24,3,2019,22,20,1,10,0
1,1,3,0,7662,1,5,2019,5,50,13,15,5
2,4,2,1,13882,9,6,2019,9,25,4,25,5
3,3,3,0,6218,12,5,2019,18,5,23,30,0
4,3,0,5,13302,1,3,2019,16,50,21,35,5
...,...,...,...,...,...,...,...,...,...,...,...,...
10678,0,3,0,4107,9,4,2019,19,55,22,25,5
10679,1,3,0,4145,27,4,2019,20,45,23,20,0
10680,4,0,2,7229,27,4,2019,8,20,11,20,0
10681,10,0,5,12648,1,3,2019,11,30,14,10,0
